In [1]:
# type annotation (review)
from typing import Optional
# better to use `None` than an empty string '' because of '' mutability
"""
Optional is not a great name, because that annotation does not
make the parameter optional. What makes it optional is assigning a
default value to the parameter. Optional[str] just means: the type
of this parameter may be str or NoneType.
"""
def show_count(count: int, singular: str, plural: Optional[str] = None) -> str:
    if count == 1:
        return f'1 {singular}'
    count_str = str(count) if count else 'no'
    if not plural:
        plural = singular + 's'
    return f'{count_str} {plural}'

In [2]:
# duck/nominal typing
class Bird:
    pass

class Duck(Bird):
    def quack(self):
        print('Quak!')

def alert(birdie):
    birdie.quack()

def alert_duck(birdie: Duck) -> None:
    birdie.quack()

def alert_bird(birdie: Bird) -> None:
    birdie.quack()

In [3]:
daffy = Duck()
alert(daffy)
alert_duck(daffy)
alert_bird(daffy)

Quak!
Quak!
Quak!


In [4]:
# Mypy would help here to catch errors (if we were to use types)
# nominal typing detects errors before runtime
woody = Bird()
alert(woody) # throws, every duck is a bird but not vise versa
alert_duck(woody)
alert_bird(woody)

AttributeError: 'Bird' object has no attribute 'quack'

In [5]:
# above we used plural: Optional[str] = None, which is shortcut for Union[str, None]
# also can be written (as of 3.10) as plural: str | None = None

# if possible avoid function that return Union types
from typing import Union
def parse_token(token: str) -> Union[str, float]:
    try:
        return float(token)
    except ValueError:
        return token

In [6]:
# Union[int, float] is redundant because int is consistent with float
# `tokenize` python >= 3.9
def tokenize(text: str) -> list[str]:
    return text.upper().split()

In [16]:
# tuples as records: tuple[str, float, str]
# tuples as named fields:
# class Coordinate(NamedTuple):
#     lat: float
#     lon: float

# tuples as immuntable sequences: tuple[str, ...] (unspecified lengths)
from collections.abc import Sequence

def columnize(sequence: Sequence[str], num_columns: int = 0) -> list[tuple[str, ...]]:
    if num_columns == 0:
        num_columns = round(len(sequence) ** 0.5)
    num_rows, remainder = divmod(len(sequence), num_columns)
    num_rows += bool(remainder)
    return [tuple(sequence[i::num_rows]) for i in range(num_rows)]

In [17]:
animals = 'drake fawn heron ibex koala lynx tahr xerus yak zapus'.split()
table = columnize(animals)
table

[('drake', 'koala', 'yak'),
 ('fawn', 'lynx', 'zapus'),
 ('heron', 'tahr'),
 ('ibex', 'xerus')]

In [18]:
import sys
import re
import unicodedata
from collections.abc import Iterator

RE_WORD = re.compile(r'\w+')
STOP_CODE = sys.maxunicode + 1

def tokenize(text: str) -> Iterator[str]:
    """return iterable of uppercased words"""
    for match in RE_WORD.finditer(text):
        yield match.group().upper()

def name_index(start: int = 32, end: int = STOP_CODE) -> dict[str, set[str]]:
    index: dict[str, set[str]] = {}
    for char in (chr(i) for i in range(start, end)):
        if name := unicodedata.name(char, ''):
            for word in tokenize(name):
                index.setdefault(word, set()).add(char)
    return index

In [22]:
index = name_index(32,65)
index['SIGN']

{'#', '$', '%', '+', '<', '=', '>'}

In [23]:
index['DIGIT']

{'0', '1', '2', '3', '4', '5', '6', '7', '8', '9'}

In [24]:
index['DIGIT'] & index['EIGHT']

{'8'}

In [25]:
index

{'SPACE': {' '},
 'EXCLAMATION': {'!'},
 'MARK': {'!', '"', '?'},
 'QUOTATION': {'"'},
 'NUMBER': {'#'},
 'SIGN': {'#', '$', '%', '+', '<', '=', '>'},
 'DOLLAR': {'$'},
 'PERCENT': {'%'},
 'AMPERSAND': {'&'},
 'APOSTROPHE': {"'"},
 'LEFT': {'('},
 'PARENTHESIS': {'(', ')'},
 'RIGHT': {')'},
 'ASTERISK': {'*'},
 'PLUS': {'+'},
 'COMMA': {','},
 'HYPHEN': {'-'},
 'MINUS': {'-'},
 'FULL': {'.'},
 'STOP': {'.'},
 'SOLIDUS': {'/'},
 'DIGIT': {'0', '1', '2', '3', '4', '5', '6', '7', '8', '9'},
 'ZERO': {'0'},
 'ONE': {'1'},
 'TWO': {'2'},
 'THREE': {'3'},
 'FOUR': {'4'},
 'FIVE': {'5'},
 'SIX': {'6'},
 'SEVEN': {'7'},
 'EIGHT': {'8'},
 'NINE': {'9'},
 'COLON': {':'},
 'SEMICOLON': {';'},
 'LESS': {'<'},
 'THAN': {'<', '>'},
 'EQUALS': {'='},
 'GREATER': {'>'},
 'QUESTION': {'?'},
 'COMMERCIAL': {'@'},
 'AT': {'@'}}

In [26]:
"""
use abc.Mapping to provide an instance of dict, defaultdict, ChainMap, and UserDict
from collections.abc import Mapping
example: Mapping[str, int], where as dict[str, int] will work but not for UserDict.
its perferred to use an abstact collection type
"""

'\nuse abc.Mapping to provide an instance of dict, defaultdict, ChainMap, and UserDict\nfrom collections.abc import Mapping\nexample: Mapping[str, int], where as dict[str, int] will work but not for UserDict.\nits perferred to use an abstact collection type\n'

In [29]:
"""
Like Sequence, Iterable is best used as a parameter type. It’s too vague as a return
type. A function should be more precise about the concrete type it returns.
"""
from collections.abc import Iterable
from typing import TypeAlias

FromTo: TypeAlias = tuple[str, str]
def zip_replace(text: str, changes: Iterable[FromTo]) -> str:
    for from_, to in changes:
        text = text.replace(from_, to)
    return text

l33t = [('a', '4'), ('e', '3'), ('i', '1'), ('o', '0')]
text = 'mad skilled noob powned leet'
zip_replace(text, l33t)

'm4d sk1ll3d n00b p0wn3d l33t'

In [30]:
# just like typescript convention we can also use [T] for generics
from collections.abc import Sequence
from random import shuffle
from typing import TypeVar

T = TypeVar('T') # this is required here as opposed to Typescript unless a lot of changes to python to accept T

def sample(population: Sequence[T], size: int) -> list[T]:
    if size < 1:
        raise ValueError('size must be >= 1')
    result = list(population)
    shuffle(result)
    return result[:size]

In [37]:
from collections import Counter
from collections.abc import Iterable, Hashable
from decimal import Decimal
from fractions import Fraction
from typing import TypeVar

NumberT = TypeVar('NumberT', float, Decimal, Fraction, str)

# this is simpler than the one in Python standard library statistics module
def mode(data: Iterable[float]) -> float:
    pairs = Counter(data).most_common(1)
    if len(pairs) == 0:
        raise ValueError('no mode for empty data')
    return pairs[0][0]

mode([1, 1, 2, 3, 3, 3, 3, 4])

# we should use generics for different types of iterables (not just floats
def mode_improved(data: Iterable[NumberT]) -> NumberT:
    pairs = Counter(data).most_common(1)
    if len(pairs) == 0:
        raise ValueError('no mode for empty data')
    return pairs[0][0]

mode_improved(["red", "blue", "blue", "red", "green", "red", "red"])

HashableT = TypeVar('HashableT', bound=Hashable)

def mode_hasable(data: Iterable[HashableT]) -> HashableT:
    pairs = Counter(data).most_common(1)
    if len(pairs) == 0:
        raise ValueError('no mode for empty data')
    return pairs[0][0]

# we can improve the type 

'red'

In [38]:
def top(series: Iterable[T], length: int) -> list[T]:
    ordered = sorted(series, reverse=True)
    return ordered[:length]

top([4,1,5,2,6,7,3], 3)
l = 'mango pear apple kiwi banana'.split()
top(l, 3)

['pear', 'mango', 'kiwi']

In [39]:
l2 = [(len(s), s) for s in l]
l2

[(5, 'mango'), (4, 'pear'), (5, 'apple'), (4, 'kiwi'), (6, 'banana')]

In [40]:
top(l2, 3)

[(6, 'banana'), (5, 'mango'), (5, 'apple')]

In [41]:
from typing import Protocol, Any
class SupportsLessThan(Protocol):
    def __lt__(self, other: Any) -> bool:
        ...

In [42]:
LT = TypeVar('LT', bound=SupportsLessThan)
def top_new(series: Iterable[LT], length: int) -> list[LT]:
    ordered = sorted(series, reverse=True)
    return ordered[:length]

In [44]:
# Callables:
from collections.abc import Callable

def update(probe: Callable[[], float], display: Callable[[float], None]) -> None:
    tempature = probe()
    display(tempature)

def probe_ok() -> int:
    return 42 # int does not break float

def display_wrong(tempature: int) -> None:
    print(hex(tempature)) # hex accepts an int but rejects a float

update(probe_ok, display_wrong) # type error

def display_ok(tempature: complex) -> None:
    print(tempature)

update(probe_ok, display_ok) # type works!

0x2a
42


In [45]:
"""
False positives:
Tools report type errors on code that is correct.

False negatives:
Tools don’t report type errors on code that is incorrect.
"""

'\nFalse positives:\nTools report type errors on code that is correct.\n\nFalse negatives:\nTools don’t report type errors on code that is incorrect.\n'

In [46]:
"""
Some important libraries skip adding types due to time, and having to support it forever
its hard to typecheck (**args) unless we type everything out. I felt kinda the same
about Typescript and just using JSDoc, focusing more on just the code, I am sure there will be
automatic processes that will analyze the code and adds types for us. Always favor tests over types
"""

'\nSome important libraries skip adding types due to time, and having to support it forever\nits hard to typecheck (**args) unless we type everything out. I felt kinda the same\nabout Typescript and just using JSDoc, focusing more on just the code, I am sure there will be\nautomatic processes that will analyze the code and adds types for us. Always favor tests over types\n'